# Extraindo dados dos exercícios das avaliações

## Dados dos exercícios

### Função que identifica a quantidade de submissões corretas

In [ ]:
def is_done(path):
    amount_correct = 0
    first_date_time = ''
    last_date_time = ''
    sum_diff_data = 0.0
    with open(path, 'r') as fl:
        is_first_date_time = True
        for line in fl:
            is_done_ = re.match(r'100\%', line)
            if is_done_: 
                amount_correct += 1
    #print('amount correct submitions:', amount_correct)
    if amount_correct > 0: 
        done = 'yes'
        #print('done: yes')
    else: 
        done = 'no'
        #print('done: no')
    return amount_correct, done

### Extraindo as datas e horas e quantidade das submissões feitas por usuário

Dados extraídos de submissões de cada exercício por usuário:

- A diferença entre a primeira data e hora e a última data e hora das submissões
- A media da diferença entre os intervalos de tempo de cada submissão por exercício
- A quantidade de submissões
- A quantidade de submissões corretas
- A quantidade de submissões incorretas

In [ ]:
def extract_subm_info(path):
    amount_correct, done = is_done(path)
    amount_submitions = 0
    first_date_time = ''
    last_date_time = ''
    sum_diff_data = 0.0
    submition_data_time = []
    with open(path, 'r') as file:
        is_first_date_time = True
        for line in file:
            submition_code = re.search(r'== SUBMITION \((.+)\)', line)
            if submition_code:
                
                #print('submition:', submition_code.group(1))
                submition_data_time.append(submition_code.group(1))
                amount_submitions += 1
                
                if is_first_date_time == True:
                    is_first_date_time = False
                    first_date_time = submition_code.group(1)#.replace('-', '/')
                    old_data = datetime.strptime(first_date_time, '%Y-%m-%d %H:%M:%S')
                
                last_date_time = submition_code.group(1)#.replace('-', '/')
                new_data = datetime.strptime(last_date_time, '%Y-%m-%d %H:%M:%S')
                if old_data <= new_data:
                    sum_diff_data += (new_data - old_data).seconds
                    old_data = new_data
                    
    #print('amount submitions:', amount_submitions)
    amount_incorrect = amount_submitions-amount_correct
    
    if first_date_time != '':
        first_date_time = datetime.strptime(first_date_time, '%Y-%m-%d %H:%M:%S')
        last_date_time = datetime.strptime(last_date_time, '%Y-%m-%d %H:%M:%S')
        data_diff = last_date_time - first_date_time
        variation_mean = sum_diff_data/amount_submitions
        variation_mean = round(variation_mean, 2)
        #print('first data submition try:', first_date_time)
        #print('last data submition try:', last_date_time)
        #print('variation data submition try:', data_diff.seconds)
        #print('mean of variations:', variation_mean)
        #values = [first_date_time.strftime('%Y-%m-%d %H:%M:%S'), 
        #         last_date_time.strftime('%Y-%m-%d %H:%M:%S'),
        values = [data_diff.seconds,
                  variation_mean, 
                  amount_submitions, 
                  amount_correct, 
                  amount_incorrect]
    else:
        #print('first data submition try:', 'NULL')
        #print('last data submition try:', 'NULL')
        #print('variation data submition try:', 'NULL')
        #print('mean of variations:', 'NULL')
        values = ['NULL','NULL', 
                  amount_submitions,
                  amount_correct,
                  amount_incorrect]
    return values, submition_data_time

### Extraindo datas e horas e quantidade testes realizados na plataforma por usuário

Dados extraídos de teste para cada exercício feito por usuário:

- A diferença entre a primeira e última data e hora de teste em segundos
- A média de tempo em segundos de todos os testes feitos
- A quantidade de testes feito pelo usuário na plataforma
- Se o exercício foi finalizado ou não

In [ ]:
def extract_test_info(path):
    amount_correct, done = is_done(path)
    amount_test = 0
    first_date_time = ''
    last_date_time = ''
    sum_diff_data = 0.0
    test_data_time = []
    with open(path, 'r') as file:
        is_first_date_time = True
        for line in file:
            test_code = re.search(r'== TEST \((.+)\)', line)
            if test_code:
                
                #print('test:', test_code.group(1))#.replace('-', '/'))
                test_data_time.append(test_code.group(1))
                amount_test += 1
                
                if is_first_date_time == True:
                    is_first_date_time = False
                    first_date_time = test_code.group(1)#.replace('-', '/')
                    old_data = datetime.strptime(first_date_time, '%Y-%m-%d %H:%M:%S')                
                
                last_date_time = test_code.group(1)#.replace('-', '/')
                new_data = datetime.strptime(last_date_time, '%Y-%m-%d %H:%M:%S')
                if old_data <= new_data:
                    sum_diff_data += (new_data - old_data).seconds
                    old_data = new_data
                                    
    #print('amount tests:',amount_test)
    
    if first_date_time != '':
        first_date_time = datetime.strptime(first_date_time, '%Y-%m-%d %H:%M:%S')
        last_date_time = datetime.strptime(last_date_time, '%Y-%m-%d %H:%M:%S')
        data_diff = last_date_time - first_date_time
        variation_mean = sum_diff_data/amount_test
        variation_mean = round(variation_mean, 2)
        #print('first data test try:', first_date_time)
       # print('last data test try:', last_date_time)
        #print('variation data test try:', data_diff.seconds)
        #print('mean of variations:', variation_mean)
        #values = [first_date_time.strftime('%Y-%m-%d %H:%M:%S'), 
        #          last_date_time.strftime('%Y-%m-%d %H:%M:%S'), 
        values = [data_diff.seconds,
                  variation_mean, 
                  amount_test, 
                  done]
    else:
        #print('first data test try:', 'NULL')
        #print('last data test try:', 'NULL')
        #print('variation data test try:', 'NULL')
        #print('mean of variations:', 'NULL')
        values = ['NULL','NULL', amount_test, done]
    return values, test_data_time


### Conectando e gravando no banco de dados

In [ ]:
import os
import re
import pymysql

from pprint import pprint
from datetime import datetime

exercise_features = [
  'exercise_id',
  'user_id',
  'assessment_id',
  'general_variation_submit',
  'mean_variations_submit',
  'amount_submit',
  'amount_correct_submit',
  'amount_incorrect_submit',
  'general_variation_test',
  'mean_variations_test',
  'amount_test',
  'done',

]

def connection_database():
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='open',
                             db='dataset-workbench',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    return connection

conn = connection_database()
cursor = conn.cursor()

cont = 0

for semester_id in os.listdir('../dataset/workbench/'):
    path0 = os.path.join('../dataset/workbench/', semester_id)
    for class_id in os.listdir(path0):
            path1 = os.path.join(path0, class_id, 'users')
            for user_id in os.listdir(path1):
                    path2 = os.path.join(path1, user_id, 'executions')
                    for execution_id in os.listdir(path2):
                        
                        path5 = os.path.join(path2, execution_id)
                        test, tests_data_time = extract_test_info(path5)
                        submit0, submitions_data_time = extract_subm_info(path5)
                        
                        assessment_id = re.sub(r'\_\d+.log', '', execution_id)
                        exercise_id = re.sub(r'\d+\_', '', execution_id)
                        exercise_id = re.sub(r'.log', '', exercise_id)
                        fk_user = f'{semester_id}#{class_id}#{user_id}'
                        fk_assessment = f'{semester_id}#{class_id}#{assessment_id}'
                        
                        values = f"{exercise_id}, '{fk_user}', '{fk_assessment}', "
                        for value in (submit0+test):
                            if type(value) == str and value != 'NULL':
                                values += f"'{value}', "
                            else: values += f"{value}, "
                            
                        features = ''
                        for feature in exercise_features:
                            features += f'{feature}, '
                        
                        insert_exercise = f"INSERT INTO executions ({features[:-2]}) VALUES ({values[:-2]}) "
                        if cont < 1000:
                            print(insert_exercise)
                            cont += 1
                        cursor.execute(insert_exercise)
                        
                        for submition in submitions_data_time:
                            features = f"user_id, assessment_id, submition_datetime, exercise_id"
                            values = f"'{fk_user}', '{fk_assessment}', '{submition}', {exercise_id}"
                            insert_submition = f"INSERT INTO submition({features}) VALUES ({values})"
                            if cont < 1000:
                                print(' ', insert_submition)
                            cursor.execute(insert_submition)
                            
                        for test in tests_data_time:
                            features = f"user_id, assessment_id, test_datetime, exercise_id"
                            values = f"'{fk_user}', '{fk_assessment}', '{test}', {exercise_id}"
                            insert_test = f"INSERT INTO test({features}) VALUES ({values})"
                            if cont < 1000:
                                print(' ', insert_test)
                            cursor.execute(insert_test)
                        
conn.commit()
conn.close()